# Week 3 - AI Lab

Author: Khushee Kapoor

Registration Number: 200968052

Setting up by importing the required libraries.

In [ ]:
# importing the libraries
import csv
import sys
import time

In this week's task, we have been given three datasets:
- people: actors mapped to unique IDs with their birth years
- movies: movies mapped to unique IDs
- stars: actors and movies mapped together basis their unique IDs

We model the above datasets as a Graph, with each actor being a Node, and the Edges representing the movies in which the actors have starred.

To assign the actors to Nodes in the graph, we create a class called Node.

In [ ]:
# intializing the Node class
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

To employ searching techniques in the Graph modelled, we use Depth First Search and Breadth First Search. For the mentioned techniques, we will use the Stack and Queue data structures. Below we have created classes with functions for the same.

In [ ]:
# stack class for depth first search
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [ ]:
# queue class for breadth first search
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

Next, we create dictionaries to store the content in the csv file.

In [ ]:
# maps names to a set of corresponding person_ids
names = {}

# maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

To fashion the graph, we determine the connecting edges between two nodes. If two actors have acted in the same movie, there is an edge between them. The below function makes a set of such edges represented as a tuple of movie ID and person ID for the passed person ID.

In [ ]:
# determining the neighbors of passed person ID
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

Since there are many actors with the same name, we make a function to resolve any unique ID ambiguities.

In [ ]:
# resolving name ambiguities
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

The below function employs the searching techniques. 

Parameters:
- method: DFS/ BFS
- source: actor 1
- target: actor 2

The function uses the passed method to determine the appropriate data structure, and then finds the shortest path between the source and target provided.

In [ ]:
# finding the shortest path between the source and target
def shortest_path(method, source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """
    solution_found = False
    no_solution = False
    solution = list()

    initial = Node(state=source, parent=None, action=None)

    if method=='bfs':
      frontier = QueueFrontier()
    if method=='dfs':
      frontier = StackFrontier()
    frontier.add(initial)
    explored = set()
    i = 0
    while solution_found == False:

        if frontier.empty() == True:
            no_solution = True
            solution_found = True
        
        node = frontier.remove()
        # print("\n\nNode in= ",node, ' i=', i)

        if node.state == target:
            # Return the solution
            solution_found = True
            while node.parent is not None:
                pid, mid = node.state, node.action
                solution.append((mid, pid))
                node = node.parent
            solution.reverse()

        explored.add(node)
        children = neighbors_for_person(node.state)
        for child in children:
            child_node = Node(state=child[1], action=child[0],parent=node)
            frontier.add(child_node)
            if child_node.state == target:
                # Return the solution
                solution_found = True
                while child_node.parent is not None:
                    pid, mid = child_node.state, child_node.action
                    solution.append((mid, pid))
                    child_node = child_node.parent
                solution.reverse()
    
    if solution_found == True:
        if no_solution == True:
            return None
        return solution

The function below reads the content in the csv files and then converts it into a dictionary as requried by the other functions.

In [ ]:
# converting the data to a dictionary
def load_data(folder):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{folder}_people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{folder}_movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{folder}_stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

The below function compiles all the above functions and calls them in the following sequence:
- load_data: to convert the csv file into a dictionary
- person_id_for_name: to resolve ambiguity of names
- shortest_path: to find the shortest path between the source and target by using dfs/ bfs

Apart from this, it takes in the following parameters:
- folder: small/ large - we use small to test for ease
- method: dfs/ bfs

If a path between the two actors exists, then the degree of connection (number of edges between them) is printed alongwith the nodes (actors) and edges (movies) involved to establish the relationship.

In [ ]:
# main function to compile the functions
def main(folder, method):

    # Load data from files into memory
    print("Loading data...")
    load_data(folder)
    print("Data loaded.")

    source = person_id_for_name(input("Enter Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Enter Name: "))
    if target is None:
        sys.exit("Person not found.")

    print(str.format('Using {}: ', method))

    path = shortest_path(method, source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

To test out the functions, we use the 'small' dataset for ease. We first use 'bfs' method.

In [17]:
start = time.time()
main('small', 'bfs')
end = time.time()
print(str.format('Elapsed time: {} seconds', end-start))

Loading data...
Data loaded.
Enter Name: Tom Hanks
Enter Name: Cary Elwes
Using bfs: 
2 degrees of separation.
1: Tom Hanks and Robin Wright starred in Forrest Gump
2: Robin Wright and Cary Elwes starred in The Princess Bride
Elapsed time: 5.847280740737915 seconds


As we can see, the bfs method can find the solution very quick. When we tried to run the same combination using dfs, it ran for 30 minutes before the notebook got disconnected. Hence, to show dfs, we have used another example of 1 degree.

In [14]:
start = time.time()
main('small', 'bfs')
end = time.time()
print(str.format('Elapsed time: {} seconds', end-start))

Loading data...
Data loaded.
Enter Name: Tom Hanks
Enter Name: Robin Wright
Using bfs: 
1 degrees of separation.
1: Tom Hanks and Robin Wright starred in Forrest Gump
Elapsed time: 5.602598190307617 seconds
